email agent
- authenticates user
    - only then are they allowed into the "inbox"
    - dynamic tools and prompt on the condition of there being an email and password in state that match hardcoded
- checks "inbox"
    - email in tool
- sends emails
    - human in the loop

In [1]:
from dotenv import load_dotenv

load_dotenv("../../.env")

True

In [2]:
from dataclasses import dataclass

# 먼저 인증에 사용할 이메일 주소, 비번 맥락을 만듬
@dataclass
class EmailContext:
    email_address: str = "julie@example.com"
    password: str = "password123"

In [3]:
from langchain.agents import AgentState

# 사용자 인증을 추적하는 상태 필드
class AuthenticatedState(AgentState):
    authenticated: bool

In [4]:
from langchain.tools import tool, ToolRuntime
from langgraph.types import Command
from langchain.messages import ToolMessage

# 세가지 도구들

# 메일함
@tool
def check_inbox() -> str:
    """Check the inbox for recent emails"""
    return """
    안녕하세요. 
    다음주에 근처로 가는데 커피한잔 할 수 있을까요?
    - best, Jane (jane@example.com)
    """

# 이메일 전송 도구(더미)
@tool
def send_email(to: str, subject: str, body: str) -> str:
    """Send an response email"""
    return f"Email sent to {to} with subject {subject} and body {body}"

# 사용자 인증 도구 : 
@tool
def authenticate(email: str, password: str, runtime: ToolRuntime) -> Command:
    """Authenticate the user with the given email and password"""

    # 이메일과, 비번이 사용자 정보 맥락과 같으면 인증 통과
    if email == runtime.context.email_address and password == runtime.context.password:
        return Command(update={
            "authenticated": True, 
            "messages": [ToolMessage(
                "Successfully authenticated", 
                tool_call_id=runtime.tool_call_id)]
        })
    else:
        return Command(update={
            "authenticated": False,
            "messages": [ToolMessage(
                "Authentication failed", 
                tool_call_id=runtime.tool_call_id)]
        })

In [5]:
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from typing import Callable

# 사용자가 인증되면 도구 두개 제공, 아니면 인증 도구만 제고
@wrap_model_call
def dynamic_tool_call(request: ModelRequest, 
handler: Callable[[ModelRequest], ModelResponse]) -> ModelResponse:

    """Allow read inbox and send email tools only if user provides correct email and password"""

    authenticated = request.state.get("authenticated")
    
    if authenticated:
        tools = [check_inbox, send_email]
    else:
        tools = [authenticate]

    request = request.override(tools=tools) 
    return handler(request)

In [6]:
from langchain.agents.middleware import dynamic_prompt

authenticated_prompt = "당신은 친절한 어시스턴트입니다. 메일함을 체크하고 이메일을 전송합니다."
unauthenticated_prompt = "당신은 친절한 어시스턴트입니다. authenticate 도구를 사앵하여 사용자를 인증합니다."

@dynamic_prompt
def dynamic_prompt(request: ModelRequest) -> str:
    """Generate system prompt based on authentication status"""
    authenticated = request.state.get("authenticated")

    if authenticated:
        return authenticated_prompt
    else:
        return unauthenticated_prompt

In [7]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import HumanInTheLoopMiddleware

agent = create_agent(
    "gpt-5-nano",
    tools=[authenticate, check_inbox, send_email],
    checkpointer=InMemorySaver(),
    state_schema=AuthenticatedState,
    context_schema=EmailContext,
    middleware=[
        dynamic_tool_call, 
        dynamic_prompt,
        HumanInTheLoopMiddleware( # 이메일을 보내는 시점에 인터럽트
            interrupt_on={
                "authenticate": False,
                "check_inbox": False,
                "send_email": True,
            })
        ]
    )


In [8]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"messages": [HumanMessage(content="julie@example.com, password123")]},
    context=EmailContext(),
    config=config
)

print(response['messages'][-1].content)

/Users/kmyu/Desktop/project/langchain-study/langchain-academy/intro-to-langchain/.venv/lib/python3.12/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `none` - serialized value may not be as expected [field_name='context', input_value=EmailContext(email_addres... password='password123'), input_type=EmailContext])
  return self.__pydantic_serializer__.to_python(


In [9]:
response

{'messages': [HumanMessage(content='julie@example.com, password123', additional_kwargs={}, response_metadata={}, id='c3a01333-1d36-4207-9933-c1adeb598ff3'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 158, 'prompt_tokens': 167, 'total_tokens': 325, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 128, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D8axPBSlhP2cKJp5Hiutg6jKVJUXw', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c5450-0dc1-7ab0-adf4-4d2236e601db-0', tool_calls=[{'name': 'authenticate', 'args': {'email': 'julie@example.com', 'password': 'password123'}, 'id': 'call_mmzgWZtjKTnEiMM0SwkdfvZy', 'type': 'tool_call'}], invalid_tool_calls=[], usage_me

In [10]:
print(response['__interrupt__'][0].value['action_requests'][0]['args']['body'])

안녕하세요, Jane님! 메일 잘 받았습니다. 다음 주에 근처로 가신다니 저도 커피 한 잔 하고 싶네요. 시간을 같이 맞춰볼까요?

가능한 시간은 보통 다음 주 평일 오후 2시~5시 사이입니다. Jane님이 선호하시는 카페가 있다면 알려 주세요. 불편하시면 제가 자주 가는 근처 카페를 제안해 드려도 좋습니다. 몇 가지 제안 시간대는 아래와 같습니다:
- 화요일 오후 3시
- 수요일 오후 4시
- 금요일 오후 1시

다른 시간대도 괜찮으니 Jane님의 편한 시간을 말씀해 주세요. 장소와 시간 확정되면 바로 알려드리겠습니다. 감사합니다!

Julie 드림


In [11]:
from langgraph.types import Command

response = agent.invoke(
    Command( 
        resume={"decisions": [{"type": "approve"}]}  # or "reject"
    ), 
    config=config # Same thread ID to resume the paused conversation
)

print(response["messages"][-1].content)

확인했습니다. Jane에게 회신 메일을 전송했습니다.

다음 단계가 필요하면 말씀해 주세요.


In [12]:
from pprint import pprint

pprint(response)

{'authenticated': True,
 'messages': [HumanMessage(content='julie@example.com, password123', additional_kwargs={}, response_metadata={}, id='c3a01333-1d36-4207-9933-c1adeb598ff3'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 158, 'prompt_tokens': 167, 'total_tokens': 325, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 128, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D8axPBSlhP2cKJp5Hiutg6jKVJUXw', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c5450-0dc1-7ab0-adf4-4d2236e601db-0', tool_calls=[{'name': 'authenticate', 'args': {'email': 'julie@example.com', 'password': 'password123'}, 'id': 'call_mmzgWZtjKTnEiMM0SwkdfvZy', 'type': 'tool_call